In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install transformers
# !pip install gradio
# !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.3 MB/s eta 0:00:00

In [3]:
# !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
# from transformers import MT5ForConditionalGeneration, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr

In [5]:
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load the fine-tuned T5 model
fine_tuned_model_path = "/soe/npullabh/244_final_project/t5/t5_fine_tuned.pt"
fine_tuned_tokenizer = "/soe/npullabh/244_final_project/t5/t5_tokenizer"
model = T5ForConditionalGeneration.from_pretrained(fine_tuned_model_path)
print(fine_tuned_tokenizer)
tokenizer = T5Tokenizer.from_pretrained(fine_tuned_tokenizer)

/soe/npullabh/244_final_project/t5/t5_tokenizer


In [4]:
# tokenizer = T5Tokenizer.from_pretrained(fine_tuned_tokenizer)
tokenizer_french = T5Tokenizer.from_pretrained(fine_tuned_tokenizer, source_lang="fr")
tokenizer_eng = T5Tokenizer.from_pretrained(fine_tuned_tokenizer, source_lang="en")

In [ ]:
# fine_tuned_model_path = "/soe/npullabh/244_final_project/t5/mt5_fine_tuned.pt"
# fine_tuned_tokenizer = "/soe/npullabh/244_final_project/t5/mt5_tokenizer"
# model = MT5ForConditionalGeneration.from_pretrained(fine_tuned_model_path)
# mt5_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_tokenizer)

In [5]:
# setup device based on availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: {}".format(device))

Using device: cuda


In [6]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [7]:
def generate_suggestions(source_sentence, partial_translation):
    # Combine the source sentence and partial translation into a single input string
    input_string = f"translate French to English: {source_sentence} <<< {partial_translation} >>>"
    print(input_string)
    # Tokenize the input string
    input_ids = tokenizer.encode(input_string, return_tensors='pt').to(device)
    # Generate the translation suggestions
    suggestions = []
    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)
        # Decode the generated suggestions and return as a list of words
        print(outputs.size())
        suggestions = tokenizer.decode(outputs[0], skip_special_tokens=True).split()[:-1]
    return suggestions

In [8]:
def generate_translation(source_text, target_prefix="", max_length=50, beam_size=7, num_suggestions=4):
    # Tokenize the source sentence and add special tokens
    input_ids = tokenizer_french.encode(source_text, return_tensors='pt')
    target_ids = tokenizer_eng.encode(target_prefix, return_tensors='pt')

    input_ids = torch.cat((input_ids, target_ids), dim=-1).to(device)

    # Generate translation suggestions using the fine-tuned T5 model
    # Experiment 1: beam search
    output = model.generate(
        input_ids=input_ids,
        num_beams=beam_size, 
        max_length=max_length,
        no_repeat_ngram_size=2,
        early_stopping=True,
        num_return_sequences=num_suggestions
    )

    suggestions = []
    word_suggestions = []
    for i in range(output.shape[0]):
        suggestion = tokenizer.decode(output[i], skip_special_tokens=True)
        suggestion = suggestion[len(target_prefix):].strip()
        suggestions.append(suggestion)

    return suggestions

In [36]:
source_text = "merci , monsieur segni , je le ferai bien volontiers"
target_prefix = ""
# thank you, mr segni, I shall do gladly
result = generate_translation("bien entendu , les américains contestent les méthodes à utiliser .", target_prefix=" of course, americans disagree on how to", beam_size=10)
print(result) # 2, 4, 6, 8, 10, 12, 14


['he methods to be used.', 'methods to be used.', 'e methods to be used.', 'to use the methods.']


In [9]:
gr.close_all()
with gr.Blocks() as demo:

    gr.Markdown("Autocomplete Demo")
    def source_update(source, translation):
        result = generate_translation(source, target_prefix="", beam_size=7, max_length=2)
        return gr.CheckboxGroup.update(choices=list(filter(lambda key: key != "", result))), gr.Textbox.update("", placeholder=" ".join(generate_suggestions(source, "")))
    
    def predict_suggestions(source, translate):

        starting_len = len(translate) + 2


        beam_size = 7
        if starting_len >= 29:
            starting_len = int(0.7 * starting_len)
            beam_size = 10

        
        result = generate_translation(source, target_prefix=translate, beam_size=beam_size, max_length=starting_len)
        result = set([" ".join(r.split(" ")[:4]) for r in result])
        return gr.CheckboxGroup.update(value=[], choices=list(filter(lambda key: len(key) > 3, result)))

    def update_translation(checkBox, translation, event: gr.SelectData):
        return gr.Textbox.update(f"{translation} {event.value}")

    with gr.Column():

        inp = gr.Textbox(placeholder="Enter Source Translation", label="Source Sentence")
        translation = gr.Textbox(placeholder="Enter Translation", label="Translation")
        checkboxGroup = gr.CheckboxGroup(choices=[], label="Suggestions", info="")
        
        
        inp.change(fn=source_update, inputs=[inp, translation], outputs=[checkboxGroup, translation])


        translation.change(fn=predict_suggestions, inputs=[inp, translation], outputs=checkboxGroup)

        checkboxGroup.select(fn=update_translation, inputs=[checkboxGroup, translation], outputs=translation)
demo.launch()
# la lega nord en italie , les vlaams blok aux pays-bas , les partisans du front national de le pen en france , constituent tous des exemples de partis ou de mouvements formés sur le thème courant de l' aversion envers les immigrants et la promotion de politiques simplistes pour les contrôler .
# it will not , as america 's racial history clearly shows .

# the lega nord in italy , the vlaams blok in the netherlands , the supporters of le pen 's national front in france , are all examples of parties or movements formed on the common theme of aversion to immigrants and promotion of simplistic policies to control them .

# bien entendu , les américains contestent les méthodes à utiliser .
# of course , americans disagree on how to do this .

# "la maison raciale divisée de l' europe"
# europe 's divided racial house

AttributeError: module 'gradio' has no attribute 'SelectData'